In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
nba_salary_file = "Resources/NBA_season1718_salary.csv"
nba_salary_df = pd.read_csv(nba_salary_file)
nba_salary_df.head()

In [ ]:
salary_cols = ["Player","Tm","season17_18"]
nba_salary = nba_salary_df[salary_cols].copy()
nba_salary = nba_salary.rename(columns = {"Tm":"Team","season17_18":"Salary"})
#nba_salary.set_index("Player", inplace = True)
nba_salary.head()


In [ ]:
nba_salary_data = nba_salary.groupby(nba_salary["Player"]).sum()
nba_salary_data

In [ ]:
nba_stats_file = "Resources/Seasons_Stats.csv"
nba_stats_data_df = pd.read_csv(nba_stats_file)
nba_stats_data_df.head()

In [ ]:
nba_stats_cols = ["Year","Player","G","FG","FGA","AST","PTS"]
nba_stats_df = nba_stats_data_df[nba_stats_cols].copy()
nba_stats_df.head()

In [ ]:
nba_career_df = nba_stats_df.groupby(nba_stats_df["Player"]).sum()
nba_career_df

In [ ]:
del nba_career_df["Year"]

In [ ]:
nba_career_df["Average_FG"]=round(nba_career_df["FG"]/nba_career_df["FGA"],2)
nba_career_df["Average_AST"]=round(nba_career_df["AST"]/nba_career_df["G"],2)
nba_career_df["Average_PTS"]=round(nba_career_df["PTS"]/nba_career_df["G"],2)

nba_career_df

In [ ]:
connection_string = 'sqlite:///player_db.sqlite'
engine = create_engine(connection_string)

In [ ]:
engine.execute('create table nba_salary_data (Player text primary key,Salary decimal(20,2))')

In [ ]:
engine.table_names()

In [ ]:
nba_salary_data.to_sql(name = 'nba_salary_data',con=engine, if_exists = 'append', index = True)

In [ ]:
pd.read_sql_query('SELECT * FROM nba_salary_data', con=engine)

In [ ]:
engine.execute('create table nba_career_data (Player text primary key,G int, FG int,FGA int,AST int, PTS int,Average_FG decimal(20,2), Average_AST decimal(20,2), Average_PTS decimal(20,2))')

In [ ]:
nba_career_df.to_sql(name = 'nba_career_data',con=engine, if_exists = 'append', index = True)

In [ ]:
pd.read_sql_query('SELECT * FROM nba_career_data', con=engine)

In [ ]:
query = """
SELECT
*
FROM nba_career_data A
INNER JOIN nba_salary_data B ON A.Player = B.Player
"""

pd.read_sql_query(query, con=engine).head()